# StackRNN

In [7]:
import keras

# 添加下面几行，否则LSTM有可能出bug
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [8]:
(x_train,y_train),(x_test,y_test)=keras.datasets.imdb.load_data(num_words=vocabulary)

In [9]:
vocabulary=10000
embedding_dim=32
word_num=500
state_dim=32

In [10]:
from keras.preprocessing import sequence
x_train=sequence.pad_sequences(x_train,maxlen=word_num)
x_test=sequence.pad_sequences(x_test,maxlen=word_num)

In [11]:
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding

model=keras.models.Sequential()
model.add(Embedding(vocabulary,embedding_dim,input_length=word_num))
model.add(LSTM(state_dim,return_sequences=True))
model.add(LSTM(state_dim,return_sequences=True))
model.add(LSTM(state_dim,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 500, 32)           8320      
_________________________________________________________________
lstm_4 (LSTM)                (None, 500, 32)           8320      
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 344,993
Trainable params: 344,993
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=1,epochs=5,batch_size=128)

Epoch 1/5
196/196 [==============================] - 30s 134ms/step - loss: 0.5450 - acc: 0.6853 - val_loss: 0.3065 - val_acc: 0.8705
Epoch 2/5
196/196 [==============================] - 26s 130ms/step - loss: 0.2248 - acc: 0.9193 - val_loss: 0.2997 - val_acc: 0.8744
Epoch 3/5
196/196 [==============================] - 25s 130ms/step - loss: 0.1590 - acc: 0.9451 - val_loss: 0.3274 - val_acc: 0.8686
Epoch 4/5
196/196 [==============================] - 26s 131ms/step - loss: 0.1190 - acc: 0.9604 - val_loss: 0.3941 - val_acc: 0.8638
Epoch 5/5
196/196 [==============================] - 26s 131ms/step - loss: 0.0941 - acc: 0.9706 - val_loss: 0.4416 - val_acc: 0.8527


In [14]:
loss,acc=model.evaluate(x_test,y_test,verbose=1)
print('Loss:%.2f, Accuracy:%.2f'%(loss,acc))

782/782 [==============================] - 19s 24ms/step - loss: 0.4416 - acc: 0.8527
Loss:0.44, Accuracy:0.85


# BiRNN

In [15]:
import keras

# 添加下面几行，否则LSTM有可能出bug
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

(x_train,y_train),(x_test,y_test)=keras.datasets.imdb.load_data(num_words=vocabulary)

vocabulary=10000
embedding_dim=32
word_num=500
state_dim=32

from keras.preprocessing import sequence
x_train=sequence.pad_sequences(x_train,maxlen=word_num)
x_test=sequence.pad_sequences(x_test,maxlen=word_num)

from keras.layers import Dense,Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding

model=keras.models.Sequential()
model.add(Embedding(vocabulary,embedding_dim,input_length=word_num))
model.add(Bidirectional(LSTM(state_dim,return_sequences=False)))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
model.summary()

model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=1,epochs=5,batch_size=128)

loss,acc=model.evaluate(x_test,y_test,verbose=1)
print('Loss:%.2f, Accuracy:%.2f'%(loss,acc))

/home/gzh/miniconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                16640     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 336,705
Trainable params: 336,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
196/196 [==============================] - 19s 89ms/step - loss: 0.6302 - acc: 0.6213 - val_loss: 0.4301 - val_acc: 0.8189
Epoch 2/5
196/196 [==============================] - 17s 87ms/step - loss: 0.3222 - acc: 0.8724 - val_loss: 0.3275 - val_acc: 0.8603
Epoch 3/5
196/196 [==============================] - 17s 87ms/step - loss: 0.2197 - acc: 0.9198 - val_loss: 0.30

# Pretrain
改进RNN->LSTM->StackLSTM->Bidirectional LSTM并没有特别大的提升，是因为Embedding的向量参数太多就存在overfitting  
只改变RNN无济于事，通过Pretrain，借助更大的语料数据集可以更好地解决这个问题。